In [1]:
import socket
import hmac
import numpy as np
from PIL import Image

def combine_pixel_groups_into_image(groups):
    combined_image = Image.new('RGB', (1979,1180 ))
    x = 0
    y = 0
    for i in sorted(groups.keys()):
        combined_image.paste(groups[i], (x, y, x + groups[i].width, y + groups[i].height))
        x += groups[i].width
        if x >= 1979:
            x = 0
            y += groups[i].height

    return combined_image

def validate(chunks):
    res = {}
    total_Verified_Data_Length = 0
    for i in chunks.keys():
         # Verify the HMAC
        if hmac.new(b'key', chunks[i][0], 'sha384').digest() != chunks[i][1]:
            res[i] = Image.new("RGB", (5, 6), (255, 0, 0))
            continue
        else:
            res[i] = chunks[i][0]
            total_Verified_Data_Length += len(chunks[i][0])
    return res, total_Verified_Data_Length

def receive_raw_image(output_path, ip, port):
    # Create a UDP socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.bind((ip, port))

    print(f"Listening on port {port} for  image...")

    chunk_size =4096
    chunks = {}
    expected_chunk_index = 0

    total_MAC_Length = 0
    total_Data_Length = 0
    total_Verified_Data_Length = 0
    Buffer =[]
    while True:
        data, addr = sock.recvfrom(chunk_size+ 4 + 48)  # 8 bytes for index + 1024 bytes of data
        if data == b'END':
            break

        # Extract the chunk index and chunk data
        chunk_index = int.from_bytes(data[:4], 'big')
        chunk_data = data[4:-48]
        mac = data[-48:]

        total_MAC_Length += len(mac)
        total_Data_Length += len(chunk_data)

        Buffer.append((chunk_index, chunk_data, mac))

        # if chunk_index%1 0:


        # if chunk_index != expected_chunk_index:
            
        #         expected_chunk_index = chunk_index

        # expected_chunk_index += 1

        



        if hmac.new(b'key', chunk_data, 'sha384').digest() != mac:
            chunks[chunk_index] = Image.new("RGB", (5, 6), (255, 0, 0))
            continue
        else:
            total_Verified_Data_Length += len(chunk_data)
            chunks[chunk_index] = Image.fromarray(np.frombuffer(chunk_data, dtype=np.uint8).reshape((6, 5, 3)))

        # chunks,total_Verified_Data_Length = validate(chunks)



       
        # print(f"Received chunk {chunk_index} from {addr}")

    for i in range(78012):
        if i not in chunks.keys():
            # print(f"Missing chunk {i}")
            chunks[i] = Image.new("RGB", (5, 6), (0, 0, 0))
            total_Data_Length += len(chunks[i].tobytes())
            total_MAC_Length += len(mac)
        # else:
        #     chunks[i] = Image.fromarray(np.frombuffer(chunk_data, dtype=np.uint8).reshape((6, 5, 3)))
        
    # Reassemble the chunks
    with open(output_path, 'wb') as f:
        for i in sorted(chunks.keys()):
            f.write(chunks[i].tobytes())


    print('message size: ',len(chunk_data)*8,',\nMAC size: ', len(mac)*8)
    return chunks , total_Verified_Data_Length/(total_Data_Length + total_MAC_Length)

# Example usage
output_path = 'received_image.NEF'
server_ip = '10.42.0.1'
listen_port = 23423
chunks, authenticated_Goodput = receive_raw_image(output_path, server_ip, listen_port)
print(f"Authenticated Goodput: {authenticated_Goodput:.2%}")
combined_image = combine_pixel_groups_into_image(chunks)

Listening on port 23423 for  image...


In [ ]:
combined_image = combine_pixel_groups_into_image(chunks)
combined_image


: 